<a href="https://colab.research.google.com/github/202422/Sales-Data-Visualization/blob/main/Transformed%20Data/SalesDataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

In [ ]:
# 1. Dossier contenant tes 12 fichiers CSV mensuels

folder_path = '/content'

In [ ]:
# 2. Liste tous les fichiers CSV dans ce dossier
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

In [ ]:
# 3. Fusionne tous les fichiers en ajoutant la provenance
dataframes = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    # Ajouter nom du fichier comme source
    df['Fichier Source'] = file
    dataframes.append(df)

In [ ]:
# 4. Combiner tous les fichiers dans un seul DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# 5. Nettoyage des données
merged_df.dropna(subset=['Order Date', 'Quantity Ordered', 'Price Each'], inplace=True)

# Supprimer lignes non valides (certaines contiennent du texte ou des entêtes dupliquées)
merged_df = merged_df[merged_df['Order Date'].str[0:1].str.isnumeric()]

# Convertir les types
merged_df['Order Date'] = pd.to_datetime(merged_df['Order Date'], format='%m/%d/%y %H:%M')
merged_df['Quantity Ordered'] = pd.to_numeric(merged_df['Quantity Ordered'])
merged_df['Price Each'] = pd.to_numeric(merged_df['Price Each'])

In [ ]:
# 6. Ajouter des colonnes utiles
merged_df['Sales'] = merged_df['Quantity Ordered'] * merged_df['Price Each']
merged_df['Month'] = merged_df['Order Date'].dt.month
merged_df['Month Name'] = merged_df['Order Date'].dt.month_name()
merged_df['Year'] = merged_df['Order Date'].dt.year
merged_df['Hour'] = merged_df['Order Date'].dt.hour

# Extraire ville et état
def extract_city(address):
    try:
        return address.split(',')[1].strip()
    except:
        return None

def extract_state(address):
    try:
        return address.split(',')[2].strip().split(' ')[0]
    except:
        return None

merged_df['City'] = merged_df['Purchase Address'].apply(extract_city)
merged_df['State'] = merged_df['Purchase Address'].apply(extract_state)
merged_df['City+State'] = merged_df['City'] + ' (' + merged_df['State'] + ')'

In [ ]:
# 7. Sauvegarder le fichier prêt pour Tableau
output_path = '/content/sales_data.csv'
output_excel_path = '/content/sales_data.xlsx'
merged_df.to_csv(output_path, index=False)
merged_df.to_excel(output_excel_path, index=False)

print(f"✅ Dataset propre enregistré ici : {output_path}")

✅ Dataset propre enregistré ici : /content/sales_data.csv
